Note: each of the cells can be run individually, as long as the prerequisite pickle files exist in `/data/processed/` directory. Otherwise, waiting for execution of all cells can take a substantial amount of time.

## Constants for the Notebook

In [66]:
file_count = 50   # TODO: change to 1000
playlists_per_file = 1000

playlist_count = file_count * playlists_per_file

## Extracting Playlist Data

The 1 000 000 spotify albums are stored in `/data/raw/playlists/` folder, downloaded from [this source](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files). The albums are stored in 1000 `.json` files with 1000 playlists each.

Read all JSON files and merge their playlist information into a single list structure. The Python list is saved into a pickle structure for intermediate representation:

Check: `playlist_list[20008]` contains a playlist with name `sleep 2`.

In [65]:
import json
import pickle

playlist_list_path = "../data/processed/playlists" + str(playlist_count) + ".pkl"

playlist_list = []

for file_index in range(file_count):
    start_playlist_id = file_index * playlists_per_file
    end_playlist_id = (file_index + 1) * playlists_per_file - 1
    playlist_json_path = "../data/raw/playlists/mpd.slice." + str(start_playlist_id) + "-" + str(end_playlist_id) + ".json"
    
    with open(playlist_json_path, 'r') as json_file:
        data = json.load(json_file)
    
    playlist_data = data.get("playlists", [])
    playlist_list += playlist_data

with open(playlist_list_path, "wb") as fout:
    pickle.dump(playlist_list, fout, protocol = pickle.HIGHEST_PROTOCOL)

print(playlist_list[20008])

{'name': 'sleep 2', 'collaborative': 'false', 'pid': 20008, 'modified_at': 1481500800, 'num_tracks': 75, 'num_albums': 66, 'num_followers': 1, 'tracks': [{'pos': 0, 'artist_name': 'Kehlani', 'track_uri': 'spotify:track:3rGew9pmFEmGD9nZ12F1tN', 'artist_uri': 'spotify:artist:0cGUm45nv7Z6M6qdXYQGTX', 'track_name': 'The Letter', 'album_uri': 'spotify:album:34JNly8P6uZRKTGnmmLNAt', 'duration_ms': 232169, 'album_name': 'You Should Be Here'}, {'pos': 1, 'artist_name': 'Justin Bieber', 'track_uri': 'spotify:track:3jHCXrZNz8fQEEf4BpWCAj', 'artist_uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s', 'track_name': 'Purpose', 'album_uri': 'spotify:album:7fZH0aUAjY3ay25obOUf2a', 'duration_ms': 210160, 'album_name': 'Purpose'}, {'pos': 2, 'artist_name': 'Molly Kate Kestner', 'track_uri': 'spotify:track:3RPDvIpJPnNC6eBuALweOD', 'artist_uri': 'spotify:artist:390x0qxDdsMmN20RAwspbf', 'track_name': 'His Daughter (Acoustic Version)', 'album_uri': 'spotify:album:2SG4AVZaZtHhzBSHP07wrr', 'duration_ms': 357153, '